# Loading and splitting Dataset
The goal of this notebook is to continue the development of the DeepTropism model using a Pytorch.<br>
The dataset was already created on the previous notebook and all the HIV-1 env V3 loop sequences where aligned.


In [18]:
# Libraries used on the analysis
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import random

First we load the Dataframe with all the sequences published on referenced articles.<br>
The D


In [19]:
df = pd.read_csv('/home/gabriel/Documents/Repos/DeepTropism/datasets/dataset_profile_final.tsv', 
                 sep='\t', names=['seq_name', 'dataset', 'label', 'sequence_aligned'])
df.head()

,seq_name,dataset,label,sequence_aligned
0,B.FR.83.HXB2_LAI_IIIB_BRU.K03455,LANL,-,CTRPNNN-TRKRI-RIQRGPGRAFVTI-----GK-IGNMRQAHC
1,A1.CD.02.LA01AlPr.KU168256,LANL,-,CIRPNNN-TRKGI-GI--GPGQTFYAA-----DAIIGNIRHAYC
2,A1.CM.08.886_24.KP718928,LANL,-,CSRPNNN-TRRSI-RI--GPGQSFYAT-----GEIIGDIREARC
3,A1.ES.15.100_117.KY496622,LANL,-,CTRPGNN-TRTSI-RI--GPGQAFYAT-----GDIIGDIRKAYC
4,A1.KE.11.DEMA111KE002.KF716474,LANL,-,CTRPNNN-TRKSV-RI--GPGQAFFAT-----GEVIGKIRKAYC


In [20]:
df.shape

(9748, 4)

In [21]:
### Check the size of the column of the alig

In [22]:
set(df['sequence_aligned'].apply(len))

{44}

### Show the original datasets present on Dataframe

In [23]:
df.dataset.value_counts()

newdb         2998
cm            2679
hivcopred     2335
geno2pheno    1188
webpssm        350
LANL           198
Name: dataset, dtype: int64

In [24]:
df = df[df.label != 'validation']
df.shape

(9748, 4)

In [25]:
df.label.value_counts()

CCR5     7705
CXCR4     937
R5X4      908
-         198
Name: label, dtype: int64

In [26]:
df

,seq_name,dataset,label,sequence_aligned
0,B.FR.83.HXB2_LAI_IIIB_BRU.K03455,LANL,-,CTRPNNN-TRKRI-RIQRGPGRAFVTI-----GK-IGNMRQAHC
1,A1.CD.02.LA01AlPr.KU168256,LANL,-,CIRPNNN-TRKGI-GI--GPGQTFYAA-----DAIIGNIRHAYC
2,A1.CM.08.886_24.KP718928,LANL,-,CSRPNNN-TRRSI-RI--GPGQSFYAT-----GEIIGDIREARC
3,A1.ES.15.100_117.KY496622,LANL,-,CTRPGNN-TRTSI-RI--GPGQAFYAT-----GDIIGDIRKAYC
4,A1.KE.11.DEMA111KE002.KF716474,LANL,-,CTRPNNN-TRKSV-RI--GPGQAFFAT-----GEVIGKIRKAYC
...,...,...,...,...
9743,DUR.AM262127.O.CCR5,cm,CCR5,CVRPGDNSVKEMRA----GPMAWYSME--LERNGSRTNSRTAFC
9744,DUR.X84327.O.CCR5,cm,CCR5,CVRPGNNSVQEIKI----GPMAWYSMQ--IEREGKGANSRTAFC
9745,CCR5_AM262114_21502_FR_1995_O,geno2pheno,CCR5,CVRPGSNSVQEIKI----GPMAWYSMQ--LEQDGKRANARTAFC
9746,CXCR4/GPR15_NDK_13796_CD_1983_D,geno2pheno,CXCR4,CTRPYKYTRQRTSI----GLRQSLYTI--TGKKKKTGYIGQAHC


## Remove samples from LANL of the Dataset

In [27]:
df_labeled = df[df.dataset != "LANL"]

In [28]:
# Function to call labels
def tropism_label(row):
    """
    Define numeric label, 'CCR5' as 0 
    and 'CXCR4' or 'R5X4' as 1
    """
    # For CCR5
    if str(row.label).strip() == 'CCR5':
        return 0
    # For CXCR4
    elif str(row.label).strip() == 'CXCR4':
        return 1
    # For R5X4
    elif str(row.label).strip() == 'R5X4':
        return 1

In [29]:
df_labeled['label_numeric'] = df_labeled.apply(tropism_label, axis=1)

/home/gabriel/miniconda3/envs/deeptropism/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_labeled

,seq_name,dataset,label,sequence_aligned,label_numeric
198,X138.EU074781.BG.CXCR4,cm,CXCR4,C-RPNN--TRKS------GPQ-----------YTIIGDIA---C,1
199,CCR5_AG1030_-_FR_-_02_AG,geno2pheno,CCR5,CSRPNNN-TRKSRI----GPGQTFYAT-----------DIGDQC,0
200,CCR5_AG1005_-_FR_-_02_AG,geno2pheno,CCR5,CTRPNNN-TRKSIH----PGRAFYATV-----------GPQAHC,0
201,-.FJ652339.02_AG.CCR5,cm,CCR5,CTRPNNNTRS--------VRIGPGQAF-------YAGDIGIQAC,0
202,-.FJ375998.C.CCR5,cm,CCR5,CRPNNTRKMR--------IGPGQTYAT-------GDIIGIRAHC,0
...,...,...,...,...,...
9743,DUR.AM262127.O.CCR5,cm,CCR5,CVRPGDNSVKEMRA----GPMAWYSME--LERNGSRTNSRTAFC,0
9744,DUR.X84327.O.CCR5,cm,CCR5,CVRPGNNSVQEIKI----GPMAWYSMQ--IEREGKGANSRTAFC,0
9745,CCR5_AM262114_21502_FR_1995_O,geno2pheno,CCR5,CVRPGSNSVQEIKI----GPMAWYSMQ--LEQDGKRANARTAFC,0
9746,CXCR4/GPR15_NDK_13796_CD_1983_D,geno2pheno,CXCR4,CTRPYKYTRQRTSI----GLRQSLYTI--TGKKKKTGYIGQAHC,1


# Filtering the dataset by unique sequences
To improve the quality of our trainning and avoid bias we are going to create a dataset with only unique sequences from the original Dataframe.
This is going to be the dataset for the development of our model based on Deep Neural Network.

In [31]:
df_unique = df_labeled.drop_duplicates(subset=['sequence_aligned'], keep='first')

In [32]:
df_unique.head()

,seq_name,dataset,label,sequence_aligned,label_numeric
198,X138.EU074781.BG.CXCR4,cm,CXCR4,C-RPNN--TRKS------GPQ-----------YTIIGDIA---C,1
199,CCR5_AG1030_-_FR_-_02_AG,geno2pheno,CCR5,CSRPNNN-TRKSRI----GPGQTFYAT-----------DIGDQC,0
200,CCR5_AG1005_-_FR_-_02_AG,geno2pheno,CCR5,CTRPNNN-TRKSIH----PGRAFYATV-----------GPQAHC,0
201,-.FJ652339.02_AG.CCR5,cm,CCR5,CTRPNNNTRS--------VRIGPGQAF-------YAGDIGIQAC,0
202,-.FJ375998.C.CCR5,cm,CCR5,CRPNNTRKMR--------IGPGQTYAT-------GDIIGIRAHC,0


In [33]:
df_unique.label_numeric.value_counts()

0    2783
1     825
Name: label_numeric, dtype: int64

In [34]:
df_unique.label.value_counts()

CCR5     2783
R5X4      485
CXCR4     340
Name: label, dtype: int64

In [35]:
df_unique.shape

(3608, 5)

## Creating Dataframes for comparing performance against other methods
In order to evaluate our model against the others already published, we are going to create separate Dataframes for each method filterinf by the 'dataset' column

In [47]:
df_labeled.dataset.value_counts()

newdb         2998
cm            2679
hivcopred     2335
geno2pheno    1188
webpssm        350
Name: dataset, dtype: int64

In [110]:
df_newdb = df_labeled[df_labeled.dataset == 'newdb']
df_cm = df_labeled[df_labeled.dataset == 'cm']
df_hivcopred = df_labeled[df_labeled.dataset == 'hivcopred']
df_geno2pheno = df_labeled[df_labeled.dataset == 'geno2pheno']
df_webpssm = df_labeled[df_labeled.dataset == 'webpssm']

In [111]:
print(df_newdb.shape)
print(df_cm.shape)
print(df_hivcopred.shape)
print(df_geno2pheno.shape)
print(df_webpssm.shape)

(2998, 5)
(2679, 5)
(2335, 5)
(1188, 5)
(350, 5)


# Spliting the Dataset for Cross Validation
We are going to create indices and set it to variables to make our cross validation reproducible. Our dataset is going to consist on:<br>
* Trainning = 80 %
* Validation = 10 %
* Test = 10 %

In [49]:
# Create a list of indices and shuffle it using seed
random.seed(42)
size = df_unique.shape[0]
list_indices = list(range(size))
random.shuffle(list_indices)

In [50]:
# Now create the list of indices for trainning, validation and test
test_indices = list_indices[:int(size/10)]
train_val_indices = list_indices[int(size/10):]

In [51]:
len(test_indices)

360

In [52]:
len(train_val_indices)

3248

In [53]:
assert len(train_val_indices) + len(test_indices) == len(list_indices), "Splitting indices with error"

## Creating the Dataloaders for trainning

In [109]:
def get_array_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    return f_array.flatten().astype(float)

In [55]:
dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}

In [45]:
len(dict_aa_pos.keys())

26

In [56]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_labeled.iterrows():
    list_data.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels.append(int(row.label_numeric))

In [57]:
len(list_data) == len(list_labels)

True

In [58]:
# For Test set
test_data = []
test_label = []
for j in test_indices:
    test_data.append(list_data[j])
    test_label.append(np.array(list_labels[j]))

test_tensor_x = torch.stack([torch.from_numpy(i) for i in test_data]) # transform to torch tensors
test_tensor_y = torch.stack([torch.from_numpy(i) for i in test_label])

test_dataset = torch.utils.data.TensorDataset(test_tensor_x,test_tensor_y) # create your test dataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=64) # create your dataloader


In [59]:
# Now we define the
len(train_val_indices)

3248

In [60]:
crossval_val_indices = np.array_split(train_val_indices,5)

In [61]:
len(crossval_val_indices[0])

650

In [62]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader

Cross Validation: 1
Cross Validation: 2
Cross Validation: 3
Cross Validation: 4
Cross Validation: 5


## Define the Deep Neural Network Architecture


In [115]:
class DeepTropism_1(nn.Module):
    def __init__(self):
        super(DeepTropism_1, self).__init__()
        self.linear1 = nn.Linear(1144,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,2)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 

model = DeepTropism_1().float()
model

DeepTropism_1(
  (linear1): Linear(in_features=1144, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=2, bias=True)
)

### For showing the metrics of the model

In [116]:
def show_metrics(y_true, y_score):
    # True positive
    tp = np.sum(y_true * y_score)
    # False positive
    fp = np.sum((y_true == 0) * y_score)
    # True negative
    tn = np.sum((y_true==0) * (y_score==0))
    # False negative
    fn = np.sum(y_true * (y_score==0))

    # True positive rate (sensitivity or recall)
    tpr = tp / (tp + fn)
    # False positive rate (fall-out)
    fpr = fp / (fp + tn)
    # Precision
    precision = tp / (tp + fp)
    # True negatvie tate (specificity)
    tnr = 1 - fpr
    # F1 score
    f1 = 2*tp / (2*tp + fp + fn)
    # ROC-AUC for binary classification
    auc = (tpr+tnr) / 2
    # MCC
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    print("True positive: ", tp)
    print("False positive: ", fp)
    print("True negative: ", tn)
    print("False negative: ", fn)

    print("True positive rate (recall): ", tpr)
    print("False positive rate: ", fpr)
    print("Precision: ", precision)
    print("True negative rate (Specificity): ", tnr)
    print("F1: ", f1)
    print("ROC-AUC: ", auc)
    print("MCC: ", mcc)

In [117]:
# Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [119]:
# For Trainning and validation set DeepTropism_1
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(400):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3248 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in validation_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy for validation set {n + 1}: {round(100.0 * correct/total, 2)}%')
                
                # Evaluate model against test set
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in test_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')

                
            running_loss = 0.0
            
            
    torch.save(model.state_dict(), f'model_cv{n+1}.ptb')
    print('Finished Training')
    

Cross Validation: 1


RuntimeError: size mismatch, m1: [1664 x 44], m2: [1144 x 250] at /tmp/pip-req-build-58y_cjjl/aten/src/TH/generic/THTensorMath.cpp:752

In [67]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        
        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy for test set: {round(100.0 * correct/total, 2)}%')

Neural Network accuracy for test set: 98.33%


In [68]:
show_metrics(labels_array, predict_array)

True positive:  44.0
False positive:  1.0
True negative:  310
False negative:  5.0
True positive rate (recall):  0.8979591836734694
False positive rate:  0.003215434083601286
Precision:  0.9777777777777777
True negative rate (Specificity):  0.9967845659163987
F1:  0.9361702127659575
ROC-AUC:  0.947371874794934
MCC:  0.9277166988984386


## Performance of DNN model against published datasets

In [69]:
df_newdb
df_cm
df_hivcopred
df_geno2pheno
df_webpssm

,seq_name,dataset,label,sequence_aligned,label_numeric
225,C.ZW.01.TC30__phen_SI,webpssm,CXCR4,CTRPGNNTIGP-------GRTFYATDR-------IIGDIRQAHC,1
263,DU179MAY00_ZA_C_SI/R5X4_u19_COETZER_(IN_PREP),webpssm,CXCR4,CTRPGNKTIRSIR-----LGPGQAFYT-------NKGDIRQASC,1
264,DU179D_ZA_C_SI/CXCR4_u19_COETZER_(IN_PREP),webpssm,CXCR4,CTRPGNKTIRSIR-----IGPGRTFYT-------NKGDIRQAYC,1
302,C.ZW.01.TC11__phen_NSI,webpssm,CCR5,CTRPNNNTRKSIW-----LGPGQAFYA-------NIIGDIRQAC,0
466,C.MW.93.960__phen_NSI,webpssm,CCR5,CTRPNNTRKSIRI-----GPGQTFYAT------NEIIGNREAHC,0
...,...,...,...,...,...
9525,C.ZA.98.TV018__phen_NSI,webpssm,CCR5,CTRPNNNTRRSMRI----RPGQTFYAT-----GEIIGDIRQAYC,0
9526,C.FR.92.FRMP130__phen_NSI,webpssm,CCR5,CTRPNNNTRRSVRI----GPGQTFYAT-----GAIIGDIRQAHC,0
9527,C.ZW.01.TC33__phen_NSI,webpssm,CCR5,CTRPNNNTRTSVRI----GPGQAFYAT-----GDIIGDIRQAHC,0
9528,C.FR.93.FRMP37__phen_NSI,webpssm,CCR5,CTRPSNNTRKSIRI----GPGQAFYAT-----NGIIGDIRAAHC,0


## For Newdb dataset

In [112]:
# Create list to append data from the df
list_data_newdb = []
list_labels_newdb = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_newdb.iterrows():
    list_data_newdb.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_newdb.append(np.array(int(row.label_numeric)))

newdb_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_newdb]) # transform to torch tensors
newdb_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_newdb])

newdb_dataset = torch.utils.data.TensorDataset(newdb_tensor_x,newdb_tensor_y) # create your test dataset
newdb_dataloader = torch.utils.data.DataLoader(newdb_dataset, batch_size=64) # create your dataloader

In [113]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in newdb_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)
print('Finished Training')

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 6 1, but got 2-dimensional input of size [64, 1144] instead

## For CM dataset

In [72]:
# Create list to append data from the df
list_data_cm = []
list_labels_cm = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_cm.iterrows():
    list_data_cm.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_cm.append(np.array(int(row.label_numeric)))

cm_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_cm]) # transform to torch tensors
cm_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_cm])

cm_dataset = torch.utils.data.TensorDataset(cm_tensor_x,cm_tensor_y) # create your test dataset
cm_dataloader = torch.utils.data.DataLoader(cm_dataset, batch_size=64) # create your dataloader

In [73]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in cm_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on test set: 93.02%
True positive:  149.0
False positive:  11.0
True negative:  2343
False negative:  176.0
True positive rate (recall):  0.4584615384615385
False positive rate:  0.004672897196261682
Precision:  0.93125
True negative rate (Specificity):  0.9953271028037384
F1:  0.6144329896907217
ROC-AUC:  0.7268943206326384
MCC:  0.6252078978894422


## For hivcopred dataset

In [74]:
# Create list to append data from the df
list_data_hivcopred = []
list_labels_hivcopred = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_hivcopred.iterrows():
    list_data_hivcopred.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_hivcopred.append(np.array(int(row.label_numeric)))

hivcopred_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_hivcopred]) # transform to torch tensors
hivcopred_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_hivcopred])

hivcopred_dataset = torch.utils.data.TensorDataset(hivcopred_tensor_x,hivcopred_tensor_y) # create your test dataset
hivcopred_dataloader = torch.utils.data.DataLoader(hivcopred_dataset, batch_size=64) # create your dataloader

In [75]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in hivcopred_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on hivcopred set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on hivcopred set: 83.94%
True positive:  202.0
False positive:  10.0
True negative:  1758
False negative:  365.0
True positive rate (recall):  0.3562610229276896
False positive rate:  0.005656108597285068
Precision:  0.9528301886792453
True negative rate (Specificity):  0.994343891402715
F1:  0.5186136071887034
ROC-AUC:  0.6753024571652022
MCC:  0.5232481860276933


## For geno2pheno dataset

In [76]:
# Create list to append data from the df
list_data_geno2pheno = []
list_labels_geno2pheno = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_geno2pheno.iterrows():
    list_data_geno2pheno.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_geno2pheno.append(np.array(int(row.label_numeric)))

geno2pheno_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_geno2pheno]) # transform to torch tensors
geno2pheno_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_geno2pheno])

geno2pheno_dataset = torch.utils.data.TensorDataset(geno2pheno_tensor_x,geno2pheno_tensor_y) # create your test dataset
geno2pheno_dataloader = torch.utils.data.DataLoader(geno2pheno_dataset, batch_size=64) # create your dataloader

In [77]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in geno2pheno_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on hivcopred set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on hivcopred set: 88.3%
True positive:  84.0
False positive:  8.0
True negative:  965
False negative:  131.0
True positive rate (recall):  0.39069767441860465
False positive rate:  0.008221993833504625
Precision:  0.9130434782608695
True negative rate (Specificity):  0.9917780061664954
F1:  0.5472312703583062
ROC-AUC:  0.69123784029255
MCC:  0.5509095303633641


## For webpssm

In [78]:
# Create list to append data from the df
list_data_webpssm = []
list_labels_webpssm = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_webpssm.iterrows():
    list_data_webpssm.append(get_array_from_sequence(str(row.sequence_aligned)))
    list_labels_webpssm.append(np.array(int(row.label_numeric)))

webpssm_tensor_x = torch.stack([torch.from_numpy(i) for i in list_data_webpssm]) # transform to torch tensors
webpssm_tensor_y = torch.stack([torch.from_numpy(i) for i in list_labels_webpssm])

webpssm_dataset = torch.utils.data.TensorDataset(webpssm_tensor_x,webpssm_tensor_y) # create your test dataset
webpssm_dataloader = torch.utils.data.DataLoader(webpssm_dataset, batch_size=64) # create your dataloader

In [79]:
correct = 0
total = 0
error = 0
labels_array = np.empty([0])
predict_array = np.empty([0])

with torch.no_grad():
    for data in webpssm_dataloader:
        images, labels = data
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)

        labels_array = np.concatenate([labels_array, labels])
        predict_array = np.concatenate([predict_array, predicted])

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        error += (predicted != labels).sum().item()

print(f'Neural Network accuracy on webpssm set: {round(100.0 * correct/total, 2)}%')
show_metrics(labels_array, predict_array)

Neural Network accuracy on webpssm set: 86.86%
True positive:  30.0
False positive:  1.0
True negative:  274
False negative:  45.0
True positive rate (recall):  0.4
False positive rate:  0.0036363636363636364
Precision:  0.967741935483871
True negative rate (Specificity):  0.9963636363636363
F1:  0.5660377358490566
ROC-AUC:  0.6981818181818182
MCC:  0.5724197297252573


# Testing LeNet Architecture

In [90]:
def get_matrix_from_sequence(protein_sequence):
    """
    Function to convert a protein sequence into a tensor.
    Each amino acid is represented by an numpy array of zeros of size 26,
    and the dict_aa_pos defines the position to be converted to 1.
    
    The function iterates over the protein sequences and stacks the arrays.
    At the end the arrays are linearized and converted to a tensor of size
    n x 26, with n the size of the protein.
    
    If the character is not present on the dict_aa_pos (eg. '-') the respective
    array is formed by zeros, and represents a missing value.
    """
    dict_aa_pos = {
    'A':1, 'R':2, 'N':3, 'D':4, 'C':5, 'Q':6, 'E':7, 'G':8,
    'H':9, 'I':10, 'L':11, 'K':12, 'M':13, 'F':14, 'P':15, 
    'O':16, 'S':17, 'U':18, 'T':19, 'W':20, 'Y':21, 'V':22, 
    'B':23, 'Z':24, 'J':25, 'X':0}
    
    f_array = np.zeros(26)
    for aa in protein_sequence:
        arr = np.zeros(26)
        if dict_aa_pos.get(aa):
            arr[dict_aa_pos.get(aa)] = 1
        f_array = np.vstack((f_array, arr))
    f_array = np.delete(f_array, 0,0)
    
    #return torch.from_numpy((f_array.flatten()).astype(float))
    return f_array.astype(float).transpose()

In [94]:
# Create list to append data from the df
list_data = []
list_labels = []

# Convert the sequences and labels to arrays to use as data on pytorch
for index, row in df_labeled.iterrows():
    list_data.append(get_matrix_from_sequence(str(row.sequence_aligned)))
    list_labels.append(int(row.label_numeric))

In [96]:
# For Trainning and validation set
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader

Cross Validation: 1
Cross Validation: 2
Cross Validation: 3
Cross Validation: 4
Cross Validation: 5


In [92]:
get_matrix_from_sequence('CTRPNNNTRRSMRI----RPGQTFYAT-----GEIIGDIRQAYC').shape

(26, 44)

In [102]:
# Defining the network (LeNet-5)  
class LeNet5(torch.nn.Module):
     
    def __init__(self):   
        super(LeNet5, self).__init__()
        # Convolution (In LeNet-5, 32x32 images are given as input. Hence padding of 2 is done below)
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(26,3), stride=1, padding=0, bias=True)
        # Max-pooling
        self.max_pool_1 = torch.nn.MaxPool2d(kernel_size=2)
        # Convolution
        self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, bias=True)
        # Max-pooling
        self.max_pool_2 = torch.nn.MaxPool2d(kernel_size=2)
        # Fully connected layer
        self.fc1 = torch.nn.Linear(16*5*5, 120)   # convert matrix with 16*5*5 (= 400) features to a matrix of 120 features (columns)
        self.fc2 = torch.nn.Linear(120, 84)       # convert matrix with 120 features to a matrix of 84 features (columns)
        self.fc3 = torch.nn.Linear(84, 2)        # convert matrix with 84 features to a matrix of 10 features (columns)
        
    def forward(self, x):
        # convolve, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.conv1(x))  
        # max-pooling with 2x2 grid
        x = self.max_pool_1(x)
        # convolve, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.conv2(x))
        # max-pooling with 2x2 grid
        x = self.max_pool_2(x)
        # first flatten 'max_pool_2_out' to contain 16*5*5 columns
        # read through https://stackoverflow.com/a/42482819/7551231
        x = x.view(-1, 16*5*5)
        # FC-1, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.fc1(x))
        # FC-2, then perform ReLU non-linearity
        x = torch.nn.functional.relu(self.fc2(x))
        # FC-3
        x = self.fc3(x)
        
        return x
     
#net = LeNet5()
model = LeNet5().float()
#model = model.float()
model

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(26, 3), stride=(1, 1))
  (max_pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (max_pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [105]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model = Net().float()
#model = model.float()
model

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [108]:
# For Trainning and validation set DeepTropism_1
# Define the cross validation indices for trainning and validation sets
crossval_val_indices = np.array_split(train_val_indices,5)

# Iterate over crossval_val_indices defining the Dataloaders
for n in range(len(crossval_val_indices)):
    print(f'Cross Validation: {n + 1}')
    trainning_data = []
    trainning_label = []
    validation_data = []
    validation_label = []

    validation_indices = list(crossval_val_indices[n])
    trainning_indices = list(set(train_val_indices) - set(validation_indices))

    for j in validation_indices:
        validation_data.append(list_data[j])
        validation_label.append(np.array(list_labels[j]))

    validation_tensor_x = torch.stack([torch.from_numpy(i) for i in validation_data]) # transform to torch tensors
    validation_tensor_y = torch.stack([torch.from_numpy(i) for i in validation_label])

    validation_dataset = torch.utils.data.TensorDataset(validation_tensor_x,validation_tensor_y) # create your test dataset
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=64) # create your dataloader
    
    for k in trainning_indices:
        trainning_data.append(list_data[k])
        trainning_label.append(np.array(list_labels[k]))

    trainning_tensor_x = torch.stack([torch.from_numpy(i) for i in trainning_data]) # transform to torch tensors
    trainning_tensor_y = torch.stack([torch.from_numpy(i) for i in trainning_label])

    trainning_dataset = torch.utils.data.TensorDataset(trainning_tensor_x,trainning_tensor_y) # create your test dataset
    trainning_dataloader = torch.utils.data.DataLoader(trainning_dataset, batch_size=64) # create your dataloader
    
    # Instantiante new model
    #model = DeepTropism_1().float()
        
    # Define Cross Validation Trainning Loop
    for epoch in range(40):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainning_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            #inputs = inputs.unsqueeze(0)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            #print(running_loss)
            #if i % 3239 == 0:    # print every 3239 mini-batches
            if epoch % 20 == 0 and i % 3248 == 0:
                print('[%d, %3d] loss: %.9f' %
                      (epoch + 1, i + 1, running_loss / 50))
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in validation_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy for validation set {n + 1}: {round(100.0 * correct/total, 2)}%')
                
                # Evaluate model against test set
                correct = 0
                total = 0
                error = 0
                labels_array = np.empty([0])
                predict_array = np.empty([0])

                with torch.no_grad():
                    for data in test_dataloader:
                        images, labels = data
                        outputs = model(images.float())
                        _, predicted = torch.max(outputs.data, 1)

                        labels_array = np.concatenate([labels_array, labels])
                        predict_array = np.concatenate([predict_array, predicted])

                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        error += (predicted != labels).sum().item()

                print(f'Neural Network accuracy on test set: {round(100.0 * correct/total, 2)}%')

                
            running_loss = 0.0
            
            
    torch.save(model.state_dict(), f'model_cv{n+1}.ptb')
    print('Finished Training')

Cross Validation: 1


RuntimeError: Expected 4-dimensional input for 4-dimensional weight 6 1 3, but got 3-dimensional input of size [64, 26, 44] instead